In [43]:
%reset -f
import importlib
import cnnClassifier.constants as const
importlib.reload(const)


[DEBUG] ROOT_DIR is: D:\ML_Pro\Chicken-Disease-Classification-Project
[DEBUG] CONFIG_FILE_PATH is: D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml
[DEBUG] PARAMS_FILE_PATH is: D:\ML_Pro\Chicken-Disease-Classification-Project\params.yaml


<module 'cnnClassifier.constants' from 'd:\\ml_pro\\chicken-disease-classification-project\\src\\cnnClassifier\\constants\\__init__.py'>

In [44]:
print(const.CONFIG_FILE_PATH)
print(const.CONFIG_FILE_PATH.exists())


D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml
True


In [45]:
import os
#CONFIG_FILE_PATH = os.path.join(os.getcwd(), "config", "config.yaml")

In [46]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
print(CONFIG_FILE_PATH)
print(CONFIG_FILE_PATH.exists())



D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml
True


In [47]:
import os 
print("Current working dir:",os.getcwd())

Current working dir: d:\


In [48]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

print("Config path:", CONFIG_FILE_PATH)
print("Config exists:", CONFIG_FILE_PATH.exists())
print("Params path:", PARAMS_FILE_PATH)
print("Params exists:", PARAMS_FILE_PATH.exists())


Config path: D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml
Config exists: True
Params path: D:\ML_Pro\Chicken-Disease-Classification-Project\params.yaml
Params exists: True


In [49]:
%pwd

'd:\\'

In [50]:
os.chdir("../")

In [51]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [52]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [53]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)

        )

        return prepare_callback_config

    
    

In [54]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [55]:
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [56]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[DEBUG] Reading YAML from: D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml
[2025-09-06 12:26:50,043: INFO: common: Attempting to load YAML from D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml]
[DEBUG] YAML content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/refs/heads/master/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_root_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'